In [2]:
import os
import torch
import numpy as np
import cv2

from torch.utils import data

from utils import recursive_glob
from augmentations import *
import random


In [59]:
import sys, os
import torch
import visdom
import argparse
import numpy as np
import logging
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

from torch.autograd import Variable
from torch.utils import data
from tqdm import tqdm
import collections

def setup_logging(name, filename=None):
    FORMAT = '%(levelname)s %(filename)s:%(lineno)4d: %(message)s'
    # Manually clear root loggers to prevent any module that may have called
    # logging.basicConfig() from blocking our logging setup
    logging.root.handlers = []
    if filename is None:
        logging.basicConfig(level=logging.INFO, format=FORMAT, stream=sys.stdout)
    else:
        logging.basicConfig(level=logging.INFO, format=FORMAT, filename=filename)
    logger = logging.getLogger(name)
    return logger     

In [61]:
if __name__ == '__main__':
    logger = setup_logging(__name__, filename='./'+'DSACPose'+'.out')

In [21]:
print(rgbd_image.color)

Image of size 804x244, with 1 channels.
Use numpy.asarray to access buffer data.


In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import random

In [49]:
class DSACPose(nn.Module):
    '''
    '''
    def __init__(self, net_capacity):
        super(DSACPose, self).__init__()

        c = net_capacity
        output_dim = 9 # rotation matrix ?

        self.global_model = False

        strides = [1, 2, 2, 2, 1, 1, 1]

        # build network
        self.conv1 = nn.Conv2d(4, 4*c, 3, strides[0], 1)
        self.bn1 = nn.BatchNorm2d(4*c)
        self.conv2 = nn.Conv2d(4*c, 8*c, 3, strides[1], 1)
        self.bn2 = nn.BatchNorm2d(8*c)
        self.conv3 = nn.Conv2d(8*c, 16*c, 3, strides[2], 1)
        self.bn3 = nn.BatchNorm2d(16*c)
        self.conv4 = nn.Conv2d(16*c, 32*c, 3, strides[3], 1)
        self.bn4 = nn.BatchNorm2d(32*c)
        self.conv5 = nn.Conv2d(32*c, 64*c, 3, strides[4], 1)
        self.bn5 = nn.BatchNorm2d(64*c)
        self.conv6 = nn.Conv2d(64*c, 64*c, 3, strides[5], 1)
        self.bn6 = nn.BatchNorm2d(64*c)
        self.conv7 = nn.Conv2d(64*c, 64*c, 3, strides[6], 1)
        self.bn7 = nn.BatchNorm2d(64*c)

        self.pool = nn.AdaptiveMaxPool2d(1) #used only for global models to support arbitrary image size
        
        self.fc1 = nn.Conv2d(64*c, 64*c, 1, 1, 0)
        self.bn8 = nn.BatchNorm2d(64*c)
        self.fc2 = nn.Conv2d(64*c, 64*c, 1, 1, 0)
        self.bn9 = nn.BatchNorm2d(64*c)
        self.fc3 = nn.Conv2d(64*c, output_dim, 1, 1, 0)

    def forward(self, input):
        '''
        Forward pass.
        
        input : 4D data tensor (BxCxHxW)
        '''
        batch_size = input.size(0)

        x = F.relu(self.bn1(self.conv1(input)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = F.relu(self.bn7(self.conv7(x)))
        
        #if self.global_model: x = self.pool(x)
        
        x = F.relu(self.bn8(self.fc1(x)))
        x = F.relu(self.bn9(self.fc2(x)))
        x = self.fc3(x)

        return x

		

In [50]:
net = DSACPose(3)
print(net)
input = torch.randn(10, 4, 244, 804) # data, channel, w, h
print(input.shape, input.dtype)
output = net(input)
print(output.shape)

DSACPose(
  (conv1): Conv2d(4, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(12, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn2): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn3): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn4): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(96, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn6): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_st

In [52]:
print()

torch.Size([10, 9, 31, 101])
